[Reference](https://medium.com/@tttgm/styling-charts-in-seaborn-92136331a541)

In [1]:
"""
Adapted from https://kaomorphism.com/2017/09/10/Sane-Date-Axes.html

Main changes: Looks at axes width rather than figure width. This allows
for better integration with charts with legends as well as FacetGrid style 
charts in seaborn.
"""

import pandas as pd
import matplotlib.dates as mdates
from datetime import timedelta as tdelta
from matplotlib.ticker import FuncFormatter

INTERVALS = {
  'YEARLY'  : [1, 2, 4, 5, 10],
  'MONTHLY' : [1, 2, 3, 4, 6],
  'DAILY'   : [1, 2],
  'WEEKLY'  : [1, 2],
  'HOURLY'  : [1, 2, 3, 4, 6, 12],
  'MINUTELY': [1, 5, 10, 15, 30],
  'SECONDLY': [1, 5, 10, 15, 30],
}

TICKS_PER_INCH = 1.5

def _next_largest(value, options):
    for i in options:
        if i >= value:
            return i
    return i

def _get_dynamic_formatter(timedelta, *fmts):
    def dynamic_formatter(x, pos):
        dx = mdates.num2date(x)
        strparts = [dx.strftime(fmt) for fmt in fmts]
        if pos > 0:
            # renders previous tick and removes common parts
            prior_dx = dx - timedelta
            prior_strparts = [prior_dx.strftime(fmt) for fmt in fmts]
            strparts = [new if new != prior else '' for new, prior in zip(strparts, prior_strparts)]
        return '\n'.join(strparts).strip()
    return dynamic_formatter


def _deduce_locators_formatters(max_ticks, data):
    data_interval_seconds = (data.max() - data.min()) / tdelta(seconds=1)
    interval_seconds = data_interval_seconds / max_ticks
    
    if interval_seconds < tdelta(minutes=0.5).total_seconds():
        # print("xticks: seconds")
        unit_multiple = _next_largest(interval_seconds, INTERVALS['SECONDLY'])
        timedelta = tdelta(seconds=unit_multiple)
        return (mdates.SecondLocator(bysecond=range(0, 60, unit_multiple)),
                FuncFormatter(_get_dynamic_formatter(timedelta, '%M%:S', '%-Hh', '%-d %b')))
    elif interval_seconds < tdelta(hours=0.5).total_seconds():
        # print("xticks: minutes")
        unit_multiple = _next_largest(interval_seconds / tdelta(minutes=1).total_seconds(), INTERVALS['MINUTELY'])
        timedelta = tdelta(minutes=unit_multiple)
        return (mdates.MinuteLocator(byminute=range(0, 60, unit_multiple)),
                FuncFormatter(_get_dynamic_formatter(timedelta, '%H%:M', '%-d %b', '%Y')))
    elif interval_seconds < tdelta(days=0.5).total_seconds():
        # print("xticks: hours")
        unit_multiple = _next_largest(interval_seconds / tdelta(hours=1).total_seconds(), INTERVALS['HOURLY'])
        timedelta = tdelta(hours=unit_multiple)
        return (mdates.HourLocator(byhour=range(0, 24, unit_multiple)),
                FuncFormatter(_get_dynamic_formatter(timedelta, '%-Hh', '%-d %b', '%Y')))
    elif interval_seconds < tdelta(days=3).total_seconds():
        # print("xticks: days")
        unit_multiple = _next_largest(interval_seconds / tdelta(days=1).total_seconds(), INTERVALS['DAILY'])
        timedelta = tdelta(days=unit_multiple)
        return (mdates.WeekdayLocator(byweekday=range(0, 7, unit_multiple)),
                FuncFormatter(_get_dynamic_formatter(timedelta, '%-d', '%b', '%Y')))
    elif interval_seconds < tdelta(days=14).total_seconds():
        # print("xticks: weeks")
        unit_multiple = _next_largest(interval_seconds / tdelta(weeks=1).total_seconds(), INTERVALS['WEEKLY'])
        timedelta = tdelta(days=unit_multiple * 7)
        return (mdates.WeekdayLocator(byweekday=0, interval=unit_multiple),
                FuncFormatter(_get_dynamic_formatter(timedelta, '%-d', '%b', '%Y')))
    elif interval_seconds < tdelta(weeks=26).total_seconds():
        # print("xticks: months")
        unit_multiple = _next_largest(interval_seconds / tdelta(weeks=4).total_seconds(), INTERVALS['MONTHLY'])
        timedelta = tdelta(weeks=unit_multiple * 4)
        return (mdates.MonthLocator(bymonth=range(1, 13, unit_multiple)),
                FuncFormatter(_get_dynamic_formatter(timedelta, '%b', '%Y')))
    else:
        # print("xticks: years")
        unit_multiple = _next_largest(interval_seconds / tdelta(weeks=52).total_seconds(), INTERVALS['YEARLY'])
        return (mdates.YearLocator(base=unit_multiple),
                mdates.DateFormatter('%Y'))
    
def nice_dates(ax):
    fig = ax.get_figure()
    
    # information for deciding tick locations
    bbox = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    xaxis_length_inch, yaxis_length_inch = bbox.width, bbox.height
    max_ticks = xaxis_length_inch * TICKS_PER_INCH
    data = pd.to_datetime(ax.lines[0].get_xdata())
    
    maj_locator, maj_formatter = _deduce_locators_formatters(max_ticks, data)

    ax.xaxis.set_major_locator(maj_locator)
    ax.xaxis.set_major_formatter(maj_formatter)

In [ ]:
fig,ax = plt.subplots(figsize=(10,8))
g = sns.barplot(
  x="cases_per100k", 
  y="Country", 
  data=df.sort_values(by="cases_per100k", ascending=False).iloc[:25],
  color=sns.color_palette()[0],
  saturation=1.,
)
# add data labels
datalabel_hbar(g)
# remove x-axis tick labels
g.xaxis.set_ticklabels([])
# rename axes labels
g.set(xlabel='Cases per 100k people', ylabel='Country')
# remove border
sns.despine(bottom=True);

In [ ]:
dff = df.groupby(['Date_reported', 'WHO_region']).agg(
  {'New_cases': 'sum'}
).reset_index()

In [ ]:
p = dff.groupby('WHO_region').apply(
  lambda x: x.set_index("Date_reported").rolling(window='7d').mean()
).reset_index().rename(columns={
  'New_cases': 'New_cases_rolling',
})

In [ ]:
# set figure asthetics
sns.set_context(rc={'lines.linewidth': 2.5})
g = sns.lineplot(
  data=p, 
  x="Date_reported", 
  y="New_cases_rolling", 
  hue="WHO_region",
)
# format dates on x-axis
nice_dates(g)
# move legend outside plot
g.legend(loc=2, bbox_to_anchor=(1.05, 1))
# # rename axes labels
g.set(xlabel="", ylabel="Daily new cases (7d rolling avg)")
# remove x gridlines
g.xaxis.grid(False)
# despine top and right borders
sns.despine(left=True, bottom=True);

In [ ]:
# restrict to just the top 8 countries by total cases
countries = df.groupby('Country')['Cumulative_cases'].max().sort_values(ascending=False).index[:8]

p = df.loc[df['Country'].isin(countries)]
p['Country_cat'] = pd.Categorical(
  p['Country'], 
  categories=countries, 
  ordered=True
)
data_sorted = p.sort_values(by='Country_cat')
# create facet grid
g = sns.FacetGrid(
  data=data_sorted, 
  col="Country", 
  col_wrap=4
)
# map the bar plot using a matplotlib plotting function
g.map(plt.bar, "Date_reported", "New_cases", color="b", alpha=0.4)
# map the lineplot using seaborn
kws_line = dict(
  x="Date_reported",
  y="New_cases_rolling",
)
g.map_dataframe(sns.lineplot, **kws_line)
# set subtitles and axis titles
g.set_titles("{col_name}")
# apply nicely formatted timeseries labels to bottom row
for ax in g.axes:
  nice_dates(ax)

In [ ]:
fig = g.get_figure()
fig.savefig('example_chart.svg', bbox_inches="tight")

In [ ]:
g.savefig('example_facet_chart.svg', bbox_inches="tight")

In [ ]:
fig = g.fig # note: fig is an attribute not a method
fig.savefig('example_facet_chart.svg', bbox_inches="tight")